In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

### 1. 保存为checkpoint

In [7]:
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

# 每个批次64张照片
batch_size = 64
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder(tf.float32, [None, 784], name='x_input')
y = tf.placeholder(tf.float32, [None, 10], name='y_input')

# 神经网络结构
W = tf.Variable(tf.truncated_normal([784,10],stddev=0.1))
b = tf.Variable(tf.zeros([10])+0.1)
prediction = tf.nn.softmax(tf.matmul(x,W) + b, name='output')

# 交叉熵代价函数
loss = tf.losses.softmax_cross_entropy(y, prediction)
# 使用Adam优化器
train_step = tf.train.AdamOptimizer(0.001).minimize(loss, name='train')

# 初始化变量
init = tf.global_variables_initializer()

#求准确率
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [8]:
# 定义一个Saver用于保存
Saver = tf.train.Saver()

with tf.Session() as sess:
    # 初始化变化
    sess.run(init)
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
        #每个周期计算准确率
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('iter: ' + str(epoch + 1) + " test accuracy: " + str(acc))
    #保存训练模型
    Saver.save(sess, 'test_ckpt_model/my_model.ckpt')

iter: 1 test accuracy: 0.9044
iter: 2 test accuracy: 0.9154
iter: 3 test accuracy: 0.9198
iter: 4 test accuracy: 0.9238
iter: 5 test accuracy: 0.9248
iter: 6 test accuracy: 0.9263
iter: 7 test accuracy: 0.9286
iter: 8 test accuracy: 0.9273
iter: 9 test accuracy: 0.9278
iter: 10 test accuracy: 0.93
iter: 11 test accuracy: 0.9281


### 2.从checkpoint恢复之同时存在.ckpt和.meta

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [19]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# 定义批次大小
batch_size = 64
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

with tf.Session() as sess:
    # 载入模型结构
    saver = tf.train.import_meta_graph('test_ckpt_model/my_model.ckpt.meta')
    # 载入模型参数
    saver.restore(sess, 'test_ckpt_model/my_model.ckpt')
    
    #根据tensor的名字获取对应的tensor
    # 之前保存模型的时候模型输出保存为output，":0"是保存模型参数时自动加上的，所以这里也要写上
    output = sess.graph.get_tensor_by_name('output:0')
    # 根据tensor的名字获取到对应的tensor
    # 之前保存模型的时候准确率计算保存为accuracy，":0"是保存模型参数时自动加上的，所以这里也要写上
    accuracy = sess.graph.get_tensor_by_name('accuracy:0')
    # 之前保存模型的时候模型训练保存为train，注意这里的train是operation不是tensor
    train_step = sess.graph.get_operation_by_name('train')
    
    # 把测试集喂到网络中计算准确率
    # x-input是模型数据的输入，":0"是保存模型参数时自动加上的，所以这里也要写上
    # y-input是模型标签的输入，":0"是保存模型参数时自动加上的，所以这里也要写上
    print(sess.run(accuracy, feed_dict={'x_input_1:0':mnist.test.images, 'y_input_1:0':mnist.test.labels}))
    
    # 在原来模型的基础上再训练11个周期
    for epoch in range(11):
        for batch in range(n_batch):
            batch_xs, batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={'x_input_1:0':batch_xs,'y_input_1:0':batch_ys})
        #准确率
        acc = sess.run(accuracy, feed_dict={'x_input_1:0':mnist.test.images, 'y_input_1:0':mnist.test.labels})
        print('iter: ' + str(epoch + 1) + ' test acc: ' + str(acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from test_ckpt_model/my_model.ckpt
0.9281
iter: 1 test acc: 0.9303
iter: 2 test acc: 0.9305
iter: 3 test acc: 0.9301
iter: 4 test acc: 0.9306
iter: 5 test acc: 0.9316
iter: 6 test acc: 0.9318
iter: 7 test acc: 0.9326
iter: 8 test acc: 0.9309
iter: 9 test acc: 0.9319
iter: 10 test acc: 0.9322
iter: 11 test acc: 0.9317


### 3.从checkpoit数据恢复方法之二 无.meta

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
# 定义批次大小
batch_size = 64
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

# 创建一个简单的神经网络，输入层784个神经元，输出层10个神经元
# 这里的模型参数需要跟之前训练好的模型参数一样
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
prediction = tf.nn.softmax(tf.matmul(x,W)+b)

# 计算准确率
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# 定义saver用来载入模型
# max_to_keep=5,在指定路径下最多保留5个模型，超过5个模型就会删除老的模型
saver = tf.train.Saver(max_to_keep=5)

# 交叉熵代价函数
loss = tf.losses.softmax_cross_entropy(y, prediction)
# 使用Adam优化器
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

# 定义会话
with tf.Session() as sess:
    # 变量初始化
    sess.run(tf.global_variables_initializer())
    # 计算测试集准确率
    print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))
    
    # 载入训练好的参数
    saver.restore(sess,'test_ckpt_model/my_model.ckpt')
    # 在此计算准确率
    print(sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels}))
    
    # 在原来模型的基础上再训练11个周期
    for epoch in range(11):
        for batch in range(n_batch):
            # 获取一个批次的数据和标签
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            # 训练模型
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        # 计算测试集准确率
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        # 打印信息
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))
        # 保存模型，global_step可以用来表示模型的训练次数或者训练周期数
        saver.save(sess, 'test_ckpt_model/my_model.ckpt', global_step=epoch)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
0.098
INFO:tensorflow:Restoring parameters from test_ckpt_model/my_model.ckpt
0.9281
Iter 0,Testing Accuracy 0.9301
Iter 1,Testing Accuracy 0.9289
Iter 2,Testing Accuracy 0.931
Iter 3,Testing Accuracy 0.9314
Iter 4,Testing Accuracy 0.9313
Iter 5,Testing Accuracy 0.9321
Iter 6,Testing Accuracy 0.9319
Iter 7,Testing Accuracy 0.933
Iter 8,Testing Accuracy 0.9315
Iter 9,Testing Accuracy 0.9327
Iter 10,Testing Accuracy 0.9333
